# 1. Download the datasets


# 2. Import the required libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from keras.utils import to_categorical
from keras.models import load_model

#3. Read Dataset and do preprocessing

In [3]:
 ds = pd.read_csv('spam.csv',delimiter=',',encoding='latin-1')
ds.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
ds.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [5]:
# Count of spam and Ham values

ds.groupby(['v1']).size()

v1
ham     4825
spam     747
dtype: int64

In [6]:
# Label Encoding target column

X = ds.v2
Y = ds.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [8]:
# Test and train split

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [9]:
# Tokenisaation function

max_words = 100
max_len =150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_test)
sequences = tok.texts_to_sequences(X_train)

sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

#4. Create Model and 5. Add Layers (LSTM, Dense-(Hidden Layers), Output)

In [10]:
# Creating LSTM model

inputs = Input(name='InputLayer',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FullyConnectedLayer1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='OutputLayer')(layer)
layer = Activation('sigmoid')(layer)

#6.Compile the model

In [13]:
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputLayer (InputLayer)     [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           5000      
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FullyConnectedLayer1 (Dense  (None, 256)              16640     
 )                                                               
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                           

#7.Fit the Model

In [14]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,validation_split=0.2)

Epoch 1/10
30/30 [==============================] - 8s 186ms/step - loss: 0.3958 - accuracy: 0.8456 - val_loss: 0.2568 - val_accuracy: 0.8861
Epoch 2/10
30/30 [==============================] - 5s 161ms/step - loss: 0.1742 - accuracy: 0.9414 - val_loss: 0.1342 - val_accuracy: 0.9557
Epoch 3/10
30/30 [==============================] - 5s 165ms/step - loss: 0.1062 - accuracy: 0.9641 - val_loss: 0.1204 - val_accuracy: 0.9599
Epoch 4/10
30/30 [==============================] - 5s 161ms/step - loss: 0.0976 - accuracy: 0.9691 - val_loss: 0.1414 - val_accuracy: 0.9451
Epoch 5/10
30/30 [==============================] - 5s 161ms/step - loss: 0.0950 - accuracy: 0.9694 - val_loss: 0.1215 - val_accuracy: 0.9641
Epoch 6/10
30/30 [==============================] - 5s 162ms/step - loss: 0.0901 - accuracy: 0.9707 - val_loss: 0.1465 - val_accuracy: 0.9420
Epoch 7/10
30/30 [==============================] - 5s 163ms/step - loss: 0.0867 - accuracy: 0.9707 - val_loss: 0.1164 - val_accuracy: 0.9684
Epoch 

#8.Save the Model

In [15]:
model.save('assignment_model')

#9.Test the model

In [16]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix  = sequence.pad_sequences(test_sequences,maxlen=max_len)


In [17]:
accuracy = model.evaluate(test_sequences_matrix,Y_test)
print('Accuracy: {:0.3f}'.format(accuracy[1]))

27/27 [==============================] - 1s 21ms/step - loss: 0.1220 - accuracy: 0.9653
Accuracy: 0.965


In [18]:
y_pred = model.predict(test_sequences_matrix)
print(y_pred[25:40].round(3))

27/27 [==============================] - 1s 14ms/step
[[0.001]
 [0.001]
 [0.001]
 [0.   ]
 [0.997]
 [0.008]
 [0.011]
 [0.001]
 [0.   ]
 [0.798]
 [0.004]
 [0.009]
 [0.011]
 [0.001]
 [0.953]]


In [19]:
print(Y_test[25:40])

[[0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]]
